In [2]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
(cross_validate,
KFold,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [68]:
np.set_printoptions(precision=4)

# 5.3.1 Validation Set Approach

In [21]:
Auto = load_data('Auto')

In [22]:
Auto_train, Auto_valid = train_test_split(Auto, 
                                         test_size=196,
                                         random_state=0)

In [23]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [24]:
summarize(results)

,coef,std err,t,P>|t|
intercept,39.9055,1.009,39.537,0.0
horsepower,-0.1563,0.009,-17.333,0.0


In [25]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2) # MSE

23.61661706966988

In [36]:
def evalMSE(terms,
            response,
            train,
            test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]

    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)

    return round(np.mean((y_test - test_pred)**2), 2)

In [37]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)): # [1, 4)
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([23.62, 18.76, 18.8 ])

In [38]:
Auto_train, Auto_valid = train_test_split(Auto,
    test_size=196,
    random_state=3)

In [39]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)): # [1, 4)
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.76, 16.95, 16.97])

# 5.3.2 Cross-Validation

### LOOCV

In [69]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
round(cv_err, 4)

24.2315

In [94]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1, 6)): # [1, 6)
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.2315, 19.2482, 19.335 , 19.4244, 19.0332])

In [71]:
A = np.array([3, 5, 9]) # adds 2 to all and add 4 to all
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

### K Fold CV

In [72]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # same splits for each degree
for i, d in enumerate(range(1, 6)): # [1, 6)
    X = np.power.outer(H, np.arange(d+1)) # gives us matrix with degrees as parameters
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.2077, 19.1853, 19.2763, 19.4785, 19.1372])

### Validation Set

In [73]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation # validation set
                        )
results['test_score']

array([23.6166])

In [75]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation # validation set
                        )
mean_score = round(results['test_score'].mean(), 4)
std_dev = round(results['test_score'].std(), 4)

mean_score, std_dev

(23.8022, 1.4218)

# 5.3.3 The Bootstrap

### Estimating the Accuracy of a Statistic of Interest

In [77]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / 
            (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

In [78]:
alpha_func(Portfolio, range(100))

0.57583207459283

In [79]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

0.6074452469619004

In [82]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [83]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE

0.09118176521277699

### Estimating the Accuracy of a Linear Regression Model

In [84]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [85]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [86]:
hp_func?

Signature:      hp_func(D, idx)
Call signature: hp_func(*args, **kwargs)
Type:           partial
String form:    functools.partial(<function boot_OLS at 0x17f3d60c0>, ModelSpec(terms=['horsepower']), 'mpg')
File:           ~/miniforge3/envs/islp/lib/python3.11/functools.py
Docstring:     
partial(func, *args, **keywords) - new function with partial application
of the given arguments and keywords.

In [88]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto,
    rng.choice(392,
    392,
    replace=True)) for _ in range(10)])

array([[39.8806, -0.1568],
       [38.733 , -0.147 ],
       [38.3173, -0.1444],
       [39.9145, -0.1578],
       [39.4335, -0.1507],
       [40.3663, -0.1591],
       [39.6233, -0.1545],
       [39.0581, -0.1495],
       [38.6669, -0.1452],
       [39.6428, -0.1556]])

In [89]:
hp_se = boot_SE(hp_func,
    Auto,
    B=1000,
    seed=10)
hp_se

intercept     0.848807
horsepower    0.007352
dtype: float64

In [90]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [91]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
    quad_model,
    'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  2.067840
poly(horsepower, degree=2, raw=True)[0]    0.033019
poly(horsepower, degree=2, raw=True)[1]    0.000120
dtype: float64

In [92]:
M = sm.OLS(Auto['mpg'],
quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64